In [1]:
from funkwpap import sympy_to_gurobi, Firm, Country, Sector, Regulator, get_emission
import sympy as sp
import sys
from gurobipy import Model, LinExpr, QuadExpr, GRB
x = sp.symbols('x')
y = sp.symbols('y')

In [2]:

# Create Sectors and Countries
Regulator1 = Regulator('EU', permit_price= 2, emission_cap= 100)
sector1 = Sector('cement', price_demand_function= 100 - 0.1*x, free_emission_multiplier= 0.1, regulator= Regulator1)
sector2 = Sector('steel', price_demand_function=150 - 0.1*x, free_emission_multiplier= 0.05, regulator= Regulator1)
sector3 = Sector('paper', price_demand_function= 200 - 0.02*x**1.5, free_emission_multiplier= 0.1, regulator= Regulator1)
country1 = Country('DE', 1, regulator= Regulator1)
country2 = Country('FI', 0.5, regulator= Regulator1)
country3 = Country('GR', size= 0.1, regulator= Regulator1)

# Create Firms using objects
firm1 = Firm('firm1', 1, 1, x*0, 0.1*sp.exp('x') + 100/y**2, 0, 0, 0, regulator= Regulator1)
firm2 = Firm('firm2', 1, 2, x*0, 5*x**3.2 + 40/y**2, 0, 0, 0, regulator= Regulator1)
firm3 = Firm('firm3', 1, 3, x*0, x**5.2 + 30/y**2, 0, 0, 0, regulator= Regulator1)
firm4 = Firm('firm4', 2, 1, x*0, x**3.3 + 20/y**2, 0, 0, 0, regulator= Regulator1)
firm5 = Firm('firm5', 2, 2, x*0, x**2.3 + 20/y**2, 0, 0, 0, regulator= Regulator1)
firm6 = Firm('firm6', 2, 3, x*0, x**3.2 + 15/y**2, 0, 0, 0, regulator= Regulator1)
firm7 = Firm('firm7', 3, 1, x*0, x**4.1 + 40/y**2, 0, 0, 0, regulator= Regulator1)
firm8 = Firm('firm8', 3, 2, x*0, x**5.1 + 50/y**2, 0, 0, 0, regulator= Regulator1)
firm9 = Firm('firm9', 3, 3, x*0, x**5 + 60/y**2, 0, 0, 0, regulator= Regulator1)


print(sector1.firms)  # List of firms in sector1
print(country1.firms)  # List of firms in country1
print(Regulator1.firm_registry)  # Dictionary of all firms

for firm in Regulator1.firm_registry.values():
    print(firm.name, firm.sector.name, firm.country.name)


[Firm(id=1, name='firm1', sector_id=1, country_id=1, actual_output=0, emission=0, profit=0), Firm(id=2, name='firm2', sector_id=1, country_id=2, actual_output=0, emission=0, profit=0), Firm(id=3, name='firm3', sector_id=1, country_id=3, actual_output=0, emission=0, profit=0)]
[Firm(id=1, name='firm1', sector_id=1, country_id=1, actual_output=0, emission=0, profit=0), Firm(id=4, name='firm4', sector_id=2, country_id=1, actual_output=0, emission=0, profit=0), Firm(id=7, name='firm7', sector_id=3, country_id=1, actual_output=0, emission=0, profit=0)]
{1: Firm(id=1, name='firm1', sector_id=1, country_id=1, actual_output=0, emission=0, profit=0), 2: Firm(id=2, name='firm2', sector_id=1, country_id=2, actual_output=0, emission=0, profit=0), 3: Firm(id=3, name='firm3', sector_id=1, country_id=3, actual_output=0, emission=0, profit=0), 4: Firm(id=4, name='firm4', sector_id=2, country_id=1, actual_output=0, emission=0, profit=0), 5: Firm(id=5, name='firm5', sector_id=2, country_id=2, actual_out

In [3]:
Regulator1.optimize_them_all(precision = 0.1, print_diff = True, BAU = True)

Set parameter Username


Academic license - for non-commercial use only - expires 2025-07-02
Max diff: 0.02697414

In [4]:
for firm in Regulator1.firm_registry.values():
    print(firm.name, firm.sector.name, firm.country.name, firm.actual_output, firm.emission, firm.profit)

firm1 cement DE 257.32525385607016 0.0 6273.590765913685
firm2 cement FI 257.32525385607016 0.0 6273.590765913685
firm3 cement GR 241.54945043799546 0.0 5888.976612559498
firm4 steel DE 376.13222749306016 0.0 13977.198356291676
firm5 steel FI 376.13222749306016 0.0 13977.198356291676
firm6 steel GR 376.13222749306016 0.0 13977.198356291676
firm7 paper DE 118.45529515647347 0.0 7785.026847447678
firm8 paper FI 118.65643010437255 0.0 7798.212598745071
firm9 paper GR 118.69719492833313 0.0 7802.412738544508


In [5]:
BAU_emissions = sum([firm.actual_output for firm in Regulator1.firm_registry.values()])
print('BAU emissions:', BAU_emissions)
for firm in Regulator1.firm_registry.values():
    firm.BAU_emission = firm.actual_output
    firm.BAU_profit = firm.profit
    

BAU emissions: 2240.4055608184954


In [6]:
Regulator1.emission_cap = 0.9*BAU_emissions
Regulator1.find_optimal_permit_price_to_meet_the_emission_cap_requirements()
print('New permit price:', Regulator1.permit_price)
Regulator1.emission_cap = 0.8*BAU_emissions
Regulator1.find_optimal_permit_price_to_meet_the_emission_cap_requirements()
print('New permit price:', Regulator1.permit_price)

Max diff: 0.098474735940Permit price: 13.671875 and total emission: 2012.1349008184884 and emission cap 2016.365004736646
New permit price: 13.671875
Max diff: 0.088354885940Permit price: 29.296875 and total emission: 1763.4306843185939 and emission cap 1792.3244486547965
New permit price: 29.296875


In [7]:
print("Firm name, Sector, Country, Actual output, Emission, Profit, BAU profit, BAU emission")
for firm in Regulator1.firm_registry.values():
    print(firm.name, firm.sector.name, firm.country.name, firm.actual_output, firm.emission, firm.profit, firm.BAU_profit, firm.BAU_emission)

Firm name, Sector, Country, Actual output, Emission, Profit, BAU profit, BAU emission
firm1 cement DE 176.04523429126368 170.40874820807892 3397.2876958554957 6273.590765913685 257.32525385607016
firm2 cement FI 187.54290076821835 186.2313963153644 3499.714481115265 6273.590765913685 257.32525385607016
firm3 cement GR 187.54290076821835 186.0326667717967 3508.92303329319 5888.976612559498 241.54945043799546
firm4 steel DE 292.0351469142523 289.47768714949615 9030.718078585052 13977.198356291676 376.13222749306016
firm5 steel FI 311.1082145879472 304.0576713997042 9681.569062855284 13977.198356291676 376.13222749306016
firm6 steel GR 311.1082145879472 308.3542571105595 9619.43183329994 13977.198356291676 376.13222749306016
firm7 paper DE 107.9130951450035 106.03274811594747 6196.524179381753 7785.026847447678 118.45529515647347
firm8 paper FI 107.9009248899625 106.37553270223167 6188.626839120727 7798.212598745071 118.65643010437255
firm9 paper GR 108.00138998938218 106.45997654541497 6

In [ ]:
Ώρα λοιπόν να φτιάξουμε ένα πείραμε για την συνέχεια εδώ. Permit Price σαν συνάρτηση του CAP ως συνάρτηση των BAU emissions

In [ ]:
# To  be filled according to the above and the other file, which is not available in this repo. 